In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage

import skimage.morphology
import skimage.segmentation
import skimage.feature

In [ ]:
# Parameters figures
plt.rcParams['figure.figsize'] = (6.0, 6.0)

# Generate an initial image with two overlapping circles

In [ ]:
x, y = np.indices((80, 80))
x1, y1, x2, y2 = 28, 28, 44, 52
r1, r2 = 16, 20
mask_circle1 = (x - x1)**2 + (y - y1)**2 < r1**2
mask_circle2 = (x - x2)**2 + (y - y2)**2 < r2**2
image = np.logical_or(mask_circle1, mask_circle2)

print(image)

f, axarr = plt.subplots(nrows=1, ncols=1, )
axarr.imshow(image, cmap='gray')
plt.show()

# Now we want to separate the two objects in image

## Generate the markers as local maxima of the distance to the background

In [ ]:
# Perfom an Exact euclidean distance transform.
distance = scipy.ndimage.distance_transform_edt(image)
print(distance)
f, axarr = plt.subplots(nrows=1, ncols=1, )
axarr.imshow(distance, cmap='gray')
plt.show()

https://docs.scipy.org/doc/scipy/reference/ndimage.html#module-scipy.ndimage

In [ ]:
local_maxi = skimage.feature.peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
                            labels=image)
f, axarr = plt.subplots(nrows=1, ncols=1, )
axarr.imshow(local_maxi, cmap='gray')
plt.show()

In [ ]:
print(np.unique(local_maxi))

In [ ]:
markers = scipy.ndimage.label(local_maxi)[0]
f, axarr = plt.subplots(nrows=1, ncols=1, )
axarr.imshow(markers, cmap='gray')
plt.show()

https://docs.scipy.org/doc/scipy/reference/ndimage.html#module-scipy.ndimage

In [ ]:
print(np.unique(markers))

In [ ]:
print(np.where(markers == 2))
print(np.where(markers == 1))

# Performing the watershed segmentation

http://scikit-image.org/docs/dev/api/skimage.morphology.html#skimage.morphology.watershed

In [ ]:
labels = skimage.segmentation.watershed(-distance, markers, mask=image)
#is the same as
#labels = skimage.morpfology.watershed(-distance, markers, mask=image)
f, axarr = plt.subplots(nrows=1, ncols=1, )
axarr.imshow(labels, cmap='gray')
plt.show()

# Plotting the results

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray, interpolation='nearest')
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray, interpolation='nearest')
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()